<a href="https://colab.research.google.com/github/DominikBurkert/PyTorch/blob/master/TransformerNetwork_Translater_ger_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Erstmal UTILS ausführen -> ToDO sollte noch in neues File ausgelagert werden

In [ ]:
pip install torchtext==0.6.0

     |████████████████████████████████| 71kB 5.9MB/s 
     |████████████████████████████████| 1.2MB 8.3MB/s 
  Found existing installation: torchtext 0.9.0
    Uninstalling torchtext-0.9.0:
      Successfully uninstalled torchtext-0.9.0


In [ ]:
import torch
from torchtext.data.metrics import bleu_score
import spacy
import sys


def translate_sentence(model, sentence, german, english, device, max_length=50):
    # Load german tokenizer
    spacy_ger = spacy.load("de")

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)

    # Go through each german token and convert to an index
    text_to_indices = [german.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    outputs = [english.vocab.stoi["<sos>"]]
    for i in range(max_length):
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        with torch.no_grad():
            output = model(sentence_tensor, trg_tensor)

        best_guess = output.argmax(2)[-1, :].item()
        outputs.append(best_guess)

        if best_guess == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    # remove start token
    return translated_sentence[1:]


def bleu(data, model, german, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)


def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

Imports

In [ ]:
"""
Created on Sat Jan 23 12:39:43 2021

@author: DBURKER
"""


import torch
import torch.nn as nn
import torch.optim as optim
import spacy #für vokabeln und tokenisierung
import os
#from utils import *
from torch.utils.tensorboard import SummaryWriter
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator



Prüfen ob Zugriff auf eine Grafikkarte vorhanden ist

In [ ]:
import tensorflow as tf
print(tf.test.gpu_device_name())
# wenn nicht zugriff auf gpu da ist -> laufzeit -> laufzeit ändern -> gpu auswählen
!nvidia-smi

/device:GPU:0
Wed Mar 10 13:04:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    58W / 149W |    124MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                          

DATA PREPROCESSING

Install the required languages

In [ ]:
!python -m spacy download de

     |████████████████████████████████| 14.9MB 1.6MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp37-none-any.whl size=14907057 sha256=6d9b94aad5bce2f8b138e8478c5cd8f6f37eded4706bfb516709b46c44455060
  Stored in directory: /tmp/pip-ephem-wheel-cache-5s2ax6kj/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [ ]:
#als erstes werden die Vokabeln geladen
spacy_ger = spacy.load("de")
spacy_eng = spacy.load("en")


In [ ]:
from google.colab import output
#I don't know why nbextensions don't work with python3.6 
with output.temporary():
  !pip install --upgrade git+https://github.com/Kreijstal/colab_inspector.git 
  !python2.7 -m pip install --upgrade git+https://github.com/blois/colab_inspector.git #yes, really.
  !jupyter nbextension install --py inspector

import inspector

# open a scratch cell (Ctrl+Alt+N)
# run there
# inspector.watch_globals()

In [ ]:

"""
DATA PREPROCESSING
"""

#Tokenizer wird angewendet
def tokenize_ger(text):
    return [tok.text for tok in spacy_ger.tokenizer(text)]

def tokenize_eng(text):
    return [tok.text for tok in spacy_eng.tokenizer(text)]

german = Field(tokenize = tokenize_ger, lower = True, init_token = "<sos>", eos_token = "<eos>")
english = Field(tokenize = tokenize_eng, lower = True, init_token = "<sos>", eos_token = "<eos>")


#load train data, valid data and test data
train_data, valid_data, test_data = Multi30k.splits(
    exts = (".de", ".en"), fields = (german, english)
)

german.build_vocab(train_data, max_size=10000, min_freq=2)
english.build_vocab(train_data, max_size=10000, min_freq=2)
#mehr infos dazu: https://torchtext.readthedocs.io/en/latest/datasets.html





training.tar.gz:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:00<00:00, 6.48MB/s]
validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 1.84MB/s]


downloading validation.tar.gz
downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 1.62MB/s]


Transformer

In [ ]:

"""

TRANSFORMER
"""
class Transformer(nn.Module):
    def __init__(
            self,
            embedding_size, #wird auf 512 gesetzt
            src_vocab_size, # = len(german.vocab)
            trg_vocab_size, # = len(english.vocab)
            src_pad_idx,
            #anzahl an heads wird für den attention-mechanismus benötigt
            num_heads, #wenn =8, dann ist die head_size = 512/8 = 64
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
            max_len, #max_len of german.vocab bzw. english.vocab
            device
        ):
            super(Transformer, self).__init__()
            #Worteinbettung = Wörter rein -> Vektoren raus
            self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
            self.src_position_embedding = nn.Embedding(max_len, embedding_size)
            self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
            self.trg_position_embedding = nn.Embedding(max_len, embedding_size)
            self.device = device
            
            #create transformer (from pytorch)
            self.transformer = nn.Transformer(
                embedding_size,
                num_heads,
                num_encoder_layers,
                num_decoder_layers,
                forward_expansion,
                dropout
            )
            self.fc_out =  nn.Linear(embedding_size, trg_vocab_size)
            self.dropout = nn.Dropout(dropout)
            self.src_pad_idx = src_pad_idx 
        
    #maskierung 
    def make_src_mask(self, src):
        #warum transpose? 
        # src shape ist: (src_len, N) aber wir brauchen (N, src_len)
        src_mask = src.transpose(0,1) == self.src_pad_idx
        #src shape: (N, src_len)
        return src_mask
    
    def forward(self, src, trg):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape
        
        src_positions = (
            torch.arange(0, src_seq_length).unsqueeze(1).expand(src_seq_length, N).to(self.device)
        )
        
        trg_positions = (
            torch.arange(0, trg_seq_length).unsqueeze(1).expand(trg_seq_length, N).to(self.device)
        )
        
        embed_src = self.dropout(
           (self.src_word_embedding(src) + self.src_position_embedding(src_positions))
        )
        
        embed_trg = self.dropout(
            (self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions))
        )
        
        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(self.device)
        
        out = self.transformer(
            embed_src,
            embed_trg,
            src_key_padding_mask = src_padding_mask,
            tgt_mask = trg_mask)
        
        
        out = self.fc_out(out)
        return out
   

In [1]:
if load_model:
    load_checkpoint(torch.load("my_ceckpoint.pth.tar"), model, optimizer)

NameError: ignored

In [ ]:
torch.save(model.state_dict(), 'checkpoint.pth')

In [ ]:
state_dict2 = torch.load('checkpoint.pth')
print(state_dict2.keys())

odict_keys(['src_word_embedding.weight', 'src_position_embedding.weight', 'trg_word_embedding.weight', 'trg_position_embedding.weight', 'transformer.encoder.layers.0.self_attn.in_proj_weight', 'transformer.encoder.layers.0.self_attn.in_proj_bias', 'transformer.encoder.layers.0.self_attn.out_proj.weight', 'transformer.encoder.layers.0.self_attn.out_proj.bias', 'transformer.encoder.layers.0.linear1.weight', 'transformer.encoder.layers.0.linear1.bias', 'transformer.encoder.layers.0.linear2.weight', 'transformer.encoder.layers.0.linear2.bias', 'transformer.encoder.layers.0.norm1.weight', 'transformer.encoder.layers.0.norm1.bias', 'transformer.encoder.layers.0.norm2.weight', 'transformer.encoder.layers.0.norm2.bias', 'transformer.encoder.layers.1.self_attn.in_proj_weight', 'transformer.encoder.layers.1.self_attn.in_proj_bias', 'transformer.encoder.layers.1.self_attn.out_proj.weight', 'transformer.encoder.layers.1.self_attn.out_proj.bias', 'transformer.encoder.layers.1.linear1.weight', 'tran

In [ ]:
ls -l


total 8
drwxr-xr-x 3 root root 4096 Mar  5 11:41 runs/
drwxr-xr-x 1 root root 4096 Mar  1 14:35 sample_data/


In [ ]:
""" 

TRAINING
"""  
#set up the training phase
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_model = True
save_model = True
        
# Training hyperparameters
num_epochs = 1
learning_rate = 3e-4
batch_size = 32

# Model Hyperparameters
src_vocab_size = len(german.vocab)
trg_vocab_size = len(english.vocab)
embedding_size = 512
num_heads  = 8
num_encoder_layers = 3
num_decoder_layers = 3
dropout = 0.10
max_len = 100
forward_expansion = 4
src_pad_idx = english.vocab.stoi["<pad>"]

#tensorboard for nice plots
writer = SummaryWriter("runs/loss_plot")
step = 0

#create the iterator
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.src),
    device=device,
 )

model = Transformer(
    embedding_size, 
    src_vocab_size, 
    trg_vocab_size, 
    src_pad_idx, 
    num_heads, 
    num_encoder_layers, 
    num_decoder_layers, 
    forward_expansion, 
    dropout, max_len, 
    device
 ).to(device)

optimizer = optim.Adam(model.parameters(), lr = learning_rate)

pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

print(model)

Transformer(
  (src_word_embedding): Embedding(7855, 512)
  (src_position_embedding): Embedding(100, 512)
  (trg_word_embedding): Embedding(5893, 512)
  (trg_position_embedding): Embedding(100, 512)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=4, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=4, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAtte

In [ ]:
if load_model:
    #load_checkpoint(torch.load("checkpoint.pth"), model, optimizer)
    load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)
    
sentence = "ein pferd geht unter einer brücke neben einem boot."

for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")
    
    if save_model:
        checkpoint = {
            "state_dict" : model.state_dict(),
            "optimizer" : optimizer.state_dict(),
        }
        
        save_checkpoint(checkpoint)
        
    model.eval()
    translated_sentence = translate_sentence(
        model, sentence, german, english, device, max_length=100
    )
    
    print(f"Translated example sentence \n {translated_sentence}")
    model.train()


    for batch_idx, batch in enumerate(train_iterator):
        inp_data = batch.src.to(device)
        target = batch.trg.to(device)
        
        #forward prop
        output = model(inp_data, target[:-1])
        
        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)
        optimizer.zero_grad()
        
        loss = criterion(output, target)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm(model.parameters(), max_norm = 1)
        
        optimizer.step()
        
        writer.add_scalar("Training loss", loss, global_step = step)
        step += 1
        
    #this is gonna take a while
    score = bleu(test_data, model, german, english, device)
    print(f"Bleu score {score*100:.2f}")